## Streaming Data Sources and Sinks
- You can create DataFrames from streaming sources using
<b>SparkSession.readStream()</b> and write the output from a result DataFrame using
<b>DataFrame.writeStream()</b>.
- In each case, you can specify the source type using the
method <b>format()</b>.

## Files
- Structured Streaming supports reading and writing data streams to and from files in
the same formats as the ones supported in batch processing: <b>plain text, CSV, JSON,
Parquet, ORC, etc.</b>

### Reading from files
- Structured Streaming can treat files written into a directory as a data stream.

In [1]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Read lines from a file stream").getOrCreate()

In [2]:
inputDirectory = 'MyInputStream/'

In [3]:
df = spark.readStream.format("text") \
    .load('MyInputStream/')

In [4]:
writer = df.writeStream.outputMode("append") \
    .format("console") \
    .option("truncate", False) \
    .option("numRows", 10)

In [5]:
query = writer.start()

22/06/10 18:14:31 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-adbc858f-1388-4db8-b292-8f761b5364bb. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
22/06/10 18:14:31 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+-------------------------------------------+
|value                                      |
+-------------------------------------------+
|DEST_COUNTRY_NAME,ORIGIN_COUNTRY_NAME,count|
|United States,Romania,1                    |
|United States,Ireland,264                  |
|United States,India,69                     |
|Egypt,United States,24                     |
|Equatorial Guinea,United States,1          |
|United States,Singapore,25                 |
|United States,Grenada,54                   |
|Costa Rica,United States,477               |
|Senegal,United States,29                   |
+-------------------------------------------+
only showing top 10 rows

-------------------------------------------
Batch: 1
-------------------------------------------
+-----+
|value|
+-----+
+-----+

-------------------------------------------
Batch: 2
-------------------------------------------
+-----

In [6]:
query.stop()

- The returned streaming DataFrame will have the specified schema. - Here are a few key points to remember when using files:
    - All the files must be of the same format and are expected to have the same schema. For example, if the format is "json" , all the files must be in the JSON format with one JSON record per line. The schema of each JSON record must match the one specified with readStream() . Violation of these assumptions can lead to incorrect parsing (e.g., unexpected null values) or query failures.
    - Each file must appear in the directory listing atomically—that is, the whole file must be available at once for reading, and once it is available, the file cannot be updated or modified. This is because Structured Streaming will process the file when the engine finds it (using directory listing) and internally mark it as processed. Any changes to that file will not be processed.
    - When there are multiple new files to process but it can only pick some of them in the next micro-batch (e.g., because of rate limits), it will select the files with the earliest timestamps. Within the micro-batch, however, there is no predefined order of reading of the selected files; all of them will be read in parallel.